In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import re

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# Sample distributions

sample_size = 100

import seaborn as sns, numpy as np
sns.set(rc={"figure.figsize": (8, 4)}); np.random.seed(0)
x = np.random.randn(sample_size)
ax = sns.distplot(x)

import seaborn as sns, numpy as np
sns.set(rc={"figure.figsize": (8, 4)}); np.random.seed(0)
x = np.random.pareto(1.2,sample_size)
ax = sns.distplot(x,)

import seaborn as sns, numpy as np
sns.set(rc={"figure.figsize": (8, 4)}); np.random.seed(0)
x = np.random.exponential(size=sample_size)
ax = sns.distplot(x)

# Choose dataset

In [ ]:
dataset_stat_file = "../correlation_experiments/maidan_pagerank_results.csv"
stat_df = pd.read_csv(dataset_stat_file, sep=" ")

#### extract number of users in data

In [ ]:
print stat_df.columns[2]

total_num_matcher = re.match(r'.*\(total=(\d+?)\)', stat_df.columns[2], re.M|re.I)
if not total_num_matcher:
    raise RuntimeError("Column name does NOT match the regex!")

#### rename a column

In [ ]:
cols = list(stat_df.columns)
cols[2] = "fraction_of_active_nodes"
stat_df.columns = cols

In [ ]:
stat_df.head()

In [ ]:
num_of_users = int(total_num_matcher.group(1))
num_of_days = len(stat_df)-1

In [ ]:
data_spearman = list(stat_df["spearman"])

# Popularity model

num_of_users = 1000
num_of_days = 20

In [ ]:
print num_of_users, num_of_days

### I. user popularity

In [ ]:
coef = 1.2
U = np.random.pareto(coef, num_of_users) 

In [ ]:
ax = sns.distplot(U)

### II. daily variations

In [ ]:
alpha = np.zeros((num_of_users, num_of_days))

for i in xrange(num_of_users):
    alpha[i,:] = np.random.exponential(scale=1.0,size=num_of_days) # scale=1.0 => mean=1 for each user 

In [ ]:
ax = sns.distplot(alpha[0,:])

### III. calculate daily centrality scores (without Markov model)

In [ ]:
X = np.zeros((num_of_users, num_of_days))

for i in xrange(num_of_users):
    for j in xrange(num_of_days):
        X[i,j] = alpha[i,j] * U[i]

In [ ]:
ax = sns.distplot(X[0,:])

### IV. Introducing Markov model

In [ ]:
p = list(stat_df["fraction_of_active_nodes"])[:num_of_days]
p_overlap = list(stat_df["fraction_of_users_in_2day_intersections"])[:num_of_days]

# Experiments

### 1.) correlations without Markov model

In [ ]:
import scipy.stats as stats

In [ ]:
pearson, spearman = [], []
for i in xrange(1,num_of_days):
    pearson.append(stats.pearsonr(X[:,i-1],X[:,i]))
    spearman.append(stats.spearmanr(X[:,i-1],X[:,i]))

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(range(num_of_days-1),pearson,'b')
plt.plot(range(num_of_days-1),spearman,'r')
plt.show()